**Author:** Benoît BAILLIF

**Purpose:** Extract used signatures from gctx files from the 2 GEO datasets 

**Input:**
- data/raw/
 - GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx
 - GSE70138_Broad_LINCS_Level5_COMPZ_n118050x12328_2017-03-06.gctx
 - GSE92742_Broad_LINCS_gene_info.txt : to find gene ids and which genes are landmarks
- data/processed/
 - sig_info_cmap : signature metadata
 - cmp_info_cmap : to find used compounds
 
**Output:** 
- data/signatures/
 - one folder per cell line

In [1]:
import os
import pandas as pd
from cmapPy.pandasGEXpress.parse import parse
from cmapPy.pandasGEXpress       import concat

# Input

In [9]:
raw_data_directory = 'data/raw/'
processed_data_directory = 'data/processed/'

sig_info_file_name = 'sig_info_cmap.csv'
sig_info_path = processed_data_directory + sig_info_file_name

cmp_info_file_name = 'cmp_info_cmap.csv'
cmp_info_path = processed_data_directory + cmp_info_file_name

GSE92742_gctx_file_name = 'GSE92742_Broad_LINCS_Level5_COMPZ.MODZ_n473647x12328.gctx'
GSE92742_gctx_path = raw_data_directory + GSE92742_gctx_file_name

GSE70138_gctx_file_name = 'GSE70138_Broad_LINCS_Level5_COMPZ_n118050x12328_2017-03-06.gctx'
GSE70138_gctx_path = raw_data_directory + GSE70138_gctx_file_name

gene_file_name = 'GSE92742_Broad_LINCS_gene_info.txt'
gene_path = raw_data_directory + gene_file_name

# Output

In [3]:
sig_directory = 'data/processed/signatures/'
if not os.path.exists(sig_directory) :
    os.makedirs(sig_directory)

# Main

In [10]:
def get_signature_gctoo(sig_id_geo, gene_ids_list, GSE92742_gctx_path, GSE70138_gctx_path) :
    """Fetch gene expression signatures GES for the given sig_ids and gene_ids.
    
    Gene expression signatures are stored on the server, in 2 gctx files.
    
    Args:
        sig_info (pd.Series) : signature information Series having
            index = sig_id : signature ID
            name = GEO : GEO origin of the signature (GSE92742 or GSE70138)
        gene_ids_list (list of str) : list of gene_ids_list to consider when fetching the GE
        GSE92742_gctx_path (str) : path to the GSE92742 gctx file
        GSE70138_gctx_path (str) : path to the GSE70138 gctx file
        
    Returns:
        gctoo : gene expression signatures corresponding to the given sig_ids
        
    """
    
    # str cast because it is the data type used by the parse function later for cid
    GSE92742_sig_ids = sig_id_geo[sig_id_geo == 'GSE92742'].index.astype(str)
    GSE70138_sig_ids = sig_id_geo[sig_id_geo == 'GSE70138'].index.astype(str)
    
    empty_GSE92742 = (len(GSE92742_sig_ids) == 0)
    empty_GSE70138 = (len(GSE70138_sig_ids) == 0)
    
    if (not empty_GSE92742) :
        GSE92742_sig_landmark = parse(GSE92742_gctx_path, rid=gene_ids_list, cid=GSE92742_sig_ids)
    if (not empty_GSE70138) :
        GSE70138_sig_landmark = parse(GSE70138_gctx_path, rid=gene_ids_list, cid=GSE70138_sig_ids)
        
    if (empty_GSE92742 & empty_GSE70138) :
        cell_line_gctoo = None
    elif (empty_GSE92742) :
        cell_line_gctoo = GSE70138_sig_landmark
    elif (empty_GSE70138) :
        cell_line_gctoo = GSE92742_sig_landmark
    else :
        cell_line_gctoo = concat.hstack([GSE92742_sig_landmark,GSE70138_sig_landmark])
        
    return(cell_line_gctoo)

## Load data and preview table

In [11]:
# The 8 selected cell_lines
cell_lines = ['MCF7', 'HA1E', 'HT29', 'A549', 'HCC515', 'PC3', 'VCAP', 'A375']

In [12]:
sig_info_cmap = pd.read_csv(sig_info_path)
sig_info_cmap = sig_info_cmap.set_index('sig_id')

In [13]:
print(sig_info_cmap.shape)
sig_info_cmap.head()

(310114, 7)


,pert_id,pert_iname,cell_id,pert_idose (µM),pert_itime (h),distil_id,GEO
sig_id,,,,,,,
AML001_CD34_24H:BRD-A03772856:0.37037,BRD-A03772856,BRD-A03772856,CD34,0.5,24,AML001_CD34_24H_X1_F1B10:J04|AML001_CD34_24H_X...,GSE92742
AML001_CD34_24H:BRD-A03772856:1.11111,BRD-A03772856,BRD-A03772856,CD34,1.0,24,AML001_CD34_24H_X1_F1B10:J03|AML001_CD34_24H_X...,GSE92742
AML001_CD34_24H:BRD-A03772856:10,BRD-A03772856,BRD-A03772856,CD34,10.0,24,AML001_CD34_24H_X1_F1B10:I03|AML001_CD34_24H_X...,GSE92742
AML001_CD34_24H:BRD-A03772856:3.33333,BRD-A03772856,BRD-A03772856,CD34,3.0,24,AML001_CD34_24H_X1_F1B10:I04|AML001_CD34_24H_X...,GSE92742
AML001_CD34_24H:BRD-A19037878:1.11111,BRD-A19037878,trichostatin-a,CD34,1.0,24,AML001_CD34_24H_X1_F1B10:F05|AML001_CD34_24H_X...,GSE92742


In [15]:
cmp_info_cmap = pd.read_csv(cmp_info_path)

In [16]:
print(cmp_info_cmap.shape)
cmp_info_cmap.head()

(21220, 7)


,pert_id,pert_iname,is_touchstone,inchi_key,canonical_smiles,pubchem_cid,used_compound
0,BRD-A00100033,nifurtimox,1.0,ARFHIAQFJWUCFH-UHFFFAOYSA-N,CC1CS(=O)(=O)CCN1N=Cc1ccc([N+](=O)[O-])o1,6842999.0,1
1,BRD-A00150179,5-hydroxytryptophan,0.0,QSHLMQDRPXXYEE-UHFFFAOYSA-N,NC(Cc1c[nH]c2cccc(O)c12)C(=O)O,589768.0,0
2,BRD-A00267231,hemado,1.0,KOCIMZNSNPOGOP-UHFFFAOYSA-N,CCCCC#Cc1nc(NC)c2ncn(C3OC(CO)C(O)C3O)c2n1,4043357.0,1
3,BRD-A00420644,SA-3676,0.0,ASCBUEVCEVGOFP-UHFFFAOYSA-N,CCN1c2ccccc2NC2N=C(OC)C(c3ccccc3)C21,2853908.0,1
4,BRD-A00474148,BRD-A00474148,0.0,RCGAUPRLRFZAMS-UHFFFAOYSA-N,O=C1Cc2cc([S+](=O)([O-])N3CCN(c4ccc(O)cc4)CC3)...,44825297.0,1


## Extraction

In [17]:
genes = pd.read_csv(gene_path, sep="\t")

In [18]:
print(genes.shape)
genes.head()

(12328, 5)


,pr_gene_id,pr_gene_symbol,pr_gene_title,pr_is_lm,pr_is_bing
0,780,DDR1,discoidin domain receptor tyrosine kinase 1,1,1
1,7849,PAX8,paired box 8,1,1
2,2978,GUCA1A,guanylate cyclase activator 1A,0,0
3,2049,EPHB3,EPH receptor B3,0,1
4,2101,ESRRA,estrogen related receptor alpha,0,1


In [19]:
landmark_genes = genes[genes['pr_is_lm'] == 1]

# Gene ID are cast to string, required for signature extraction by cmappy package
landmark_genes['pr_gene_id'] = landmark_genes['pr_gene_id'].astype(str)

gene_id_to_name = landmark_genes.set_index('pr_gene_id')['pr_gene_symbol']
landmark_genes_list = landmark_genes['pr_gene_id'].tolist()

/home/bens/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [20]:
# Extract 10 µM / 24 h signatures for each cell line and mean per pert_id

dose_is_10um = sig_info_cmap['pert_idose (µM)'] == 10
time_is_24h = sig_info_cmap['pert_itime (h)'] == 24
sig_info_10um_24h = sig_info_cmap[dose_is_10um & time_is_24h]

In [21]:
for cell_line in cell_lines :

    print(cell_line + ' signatures extraction')
    
    sig_info_cell_10um_24h = sig_info_10um_24h[sig_info_10um_24h['cell_id'] == cell_line]
    sig_ids_geo = sig_info_cell_10um_24h['GEO']
    cell_line_gctoo = get_signature_gctoo(sig_ids_geo, landmark_genes_list, GSE92742_gctx_path,
                                                  GSE70138_gctx_path)

    if cell_line_gctoo != None :

        # transposed to get cmap instance as rows
        cell_line_df = cell_line_gctoo.data_df.transpose()
        
        # sort columns to have the same order between signatures coming from different cell lines
        # later in the workflow
        cell_line_df = cell_line_df.reindex(sorted(cell_line_df.columns), axis=1)
        
        print('Total signatures shape (with multiple signatures per perturbagen): ' + cell_line_df.shape)
        
        cell_line_df = cell_line_df.rename(gene_id_to_name, axis=1)
        cell_line_df = cell_line_df.join(sig_info_cell_10um_24h['pert_id'], how='inner')
        cell_line_df_pert_id_meaned = cell_line_df.groupby('pert_id').mean()

        print('Total perturbagen meaned signatures: ' + cell_line_df_pert_id_meaned.shape)
        
        cell_line_df_pert_id_meaned.to_csv(sig_directory + cell_line + '_used_signatures.csv')

MCF7 signatures extraction
(10195, 978)
(7546, 978)
MCF7 done

HA1E signatures extraction
(4400, 978)
(3646, 978)
HA1E done

HT29 signatures extraction
(3605, 978)
(3192, 978)
HT29 done

A549 signatures extraction
(6069, 978)
(5267, 978)
A549 done

HCC515 signatures extraction
(2098, 978)
(1932, 978)
HCC515 done

PC3 signatures extraction
(10314, 978)
(8071, 978)
PC3 done

VCAP signatures extraction
(7554, 978)
(6365, 978)
VCAP done

A375 signatures extraction
(3962, 978)
(3525, 978)
A375 done

